In [1]:
import pandas as pd
df=pd.read_csv("C:\\Users\\yasha\\Downloads\\npr.csv")

In [2]:
df.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vcv=CountVectorizer(max_df=0.95,min_df=2,stop_words='english')

In [8]:
dtm=cv.fit_transform(df['Article'])

In [9]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

# NME

In [11]:
from sklearn.decomposition import NMF

In [12]:
nmf=NMF(n_components=7,random_state=42)

In [13]:
# This can take a while, we're dealing with a large amount of documents!
nmf.fit(dtm)

NMF(n_components=7, random_state=42)

In [15]:
len(nmf.components_)

7

In [16]:
print(nmf.components_[0])

[0.02593076 0.         0.00049873 ... 0.00118791 0.0012006  0.00296506]


In [17]:
len(nmf.components_[0])

54777

# Displaying Topics

In [21]:
len(cv.get_feature_names_out())

54777

In [22]:
import random

In [23]:
for i in range(10):
    random_word_id=random.randint(0,54776)
    print(cv.get_feature_names_out()[random_word_id])

srinivasa
technicalities
deployed
ddos
rapidly
oaxaca
needham
nibble
excludes
segall


In [24]:
single_topic=nmf.components_[0]

In [25]:
# Returns the indices that would sort this array 
single_topic.argsort()

array([33703, 41834, 41835, ..., 26752, 36283, 28659], dtype=int64)

In [27]:
# Word least representative of this topic
single_topic[42993]

0.0

In [29]:
# Top 10 words for this topic
single_topic.argsort()[-10:]

array([15008, 53211, 49459, 21073, 39848, 27439, 49183, 26752, 36283,
       28659], dtype=int64)

In [30]:
top_word_indices=single_topic.argsort()[-10:]

In [32]:
for index in top_word_indices:
    print(cv.get_feature_names_out()[index])

don
way
time
going
really
know
think
just
people
like


In [34]:
for index,topic in enumerate(nmf.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'lot', 'make', 'things', 've', 'don', 'way', 'time', 'going', 'really', 'know', 'think', 'just', 'people', 'like']


THE TOP 15 WORDS FOR TOPIC #1
['presidential', 'republicans', 'comey', 'administration', 'obama', 'new', 'election', 'republican', 'white', 'donald', 'house', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['government', 'say', 'city', 'day', 'university', 'year', 'water', 'food', 'time', 'just', 'new', 'years', 'like', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['according', 'reports', 'news', 'department', 'new', 'law', 'time', 'government', 'house', 'npr', 'court', 'told', 'president', 'police', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['country', 'election', 'new', 'states', 'vote', 'party', 'obama', 'democratic', 'voters', 'percent', 'campaign', 'hillary', 'state', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['new', 'percent', 'year', 'parents', 'private', 'kids', 'children', 'student',

# Attaching Discovered Topic Labels to Original Articles

In [35]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [36]:
dtm.shape

(11992, 54777)

In [38]:
len(df)

11992

In [39]:
topic_results=nmf.transform(dtm)

In [40]:
topic_results.shape

(11992, 7)

In [41]:
topic_results[0]

array([0.05367521, 0.88498002, 0.00660074, 0.24558176, 0.18697022,
       0.01331588, 0.        ])

In [42]:
topic_results[0].round(2)

array([0.05, 0.88, 0.01, 0.25, 0.19, 0.01, 0.  ])

In [43]:
topic_results[0].argmax()

1

# Combining with original data

In [44]:
df.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [45]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 2], dtype=int64)

In [46]:
df['Topic']=topic_results.argmax(axis=1)

In [48]:
df.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",5
5,I did not want to join yoga class. I hated tho...,0
6,With a who has publicly supported the debunk...,6
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",2
9,"Eighteen years ago, on New Year’s Eve, David F...",2


In [ ]:
# Glove